# Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

/home/fanpeng/miniconda3/envs/tf2/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-03-18 15:41:16.365490: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-18 15:41:16.449199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-18 15:41:16.449528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Questions

## 1. 
What are the advantages of a CNN over a fully connected DNN for image classi‐ fication?


These are the main advantages of a CNN over a fully connected DNN for image classification:

- Because consecutive layers are only partially connected and because it heavily
reuses its weights, a CNN has many fewer parameters than a fully connected
DNN, which makes it much faster to train, reduces the risk of overfitting, and
requires much less training data.

- When a CNN has learned a kernel that can detect a particular feature, it can
detect that feature anywhere in the image. In contrast, when a DNN learns a
feature in one location, it can detect it only in that particular location. Since
images typically have very repetitive features, CNNs are able to generalize
much better than DNNs for image processing tasks such as classification, using
fewer training examples.

- Finally, a DNN has no prior knowledge of how pixels are organized; it does not
know that nearby pixels are close. A CNN’s architecture embeds this prior
knowledge. Lower layers typically identify features in small areas of the images,
while higher layers combine the lower-level features into larger features. This
works well with most natural images, giving CNNs a decisive head start com‐
pared to DNNs.

## 2. 
Consider a CNN composed of three convolutional layers, each with 3 × 3 kernels, a stride of 2, and "same" padding. The lowest layer outputs 100 feature maps, the middle one outputs 200, and the top one outputs 400. The input images are RGB images of 200 × 300 pixels. What is the total number of parameters in the CNN? If we are using 32-bit floats, at least how much RAM will this network require when making a prediction for a single instance? What about when training on a mini-batch of 50 images?


Let’s compute how many parameters the CNN has. Since its first convolutional
layer has 3 × 3 kernels, and the input has three channels (red, green, and blue),
each feature map has 3 × 3 × 3 weights, plus a bias term. That’s 28 parameters per
739feature map. Since this first convolutional layer has 100 feature maps, it has a
total of 2,800 parameters. The second convolutional layer has 3 × 3 kernels and
its input is the set of 100 feature maps of the previous layer, so each feature map
has 3 × 3 × 100 = 900 weights, plus a bias term. Since it has 200 feature maps, this
layer has 901 × 200 = 180,200 parameters. Finally, the third and last convolu‐
tional layer also has 3 × 3 kernels, and its input is the set of 200 feature maps of
the previous layers, so each feature map has 3 × 3 × 200 = 1,800 weights, plus a
bias term. Since it has 400 feature maps, this layer has a total of 1,801 × 400 =
720,400 parameters. All in all, the CNN has 2,800 + 180,200 + 720,400 = 903,400
parameters.

Now let’s compute how much RAM this neural network will require (at least)
when making a prediction for a single instance. First let’s compute the feature
map size for each layer. Since we are using a stride of 2 and "same" padding, the
horizontal and vertical dimensions of the feature maps are divided by 2 at each
layer (rounding up if necessary). So, as the input channels are 200 × 300 pixels,
the first layer’s feature maps are 100 × 150, the second layer’s feature maps are 50
× 75, and the third layer’s feature maps are 25 × 38. Since 32 bits is 4 bytes and
the first convolutional layer has 100 feature maps, this first layer takes up 4 × 100
× 150 × 100 = 6 million bytes (6 MB). The second layer takes up 4 × 50 × 75 ×
200 = 3 million bytes (3 MB). Finally, the third layer takes up 4 × 25 × 38 × 400 =
1,520,000 bytes (about 1.5 MB). However, once a layer has been computed, the
memory occupied by the previous layer can be released, so if everything is well
optimized, only 6 + 3 = 9 million bytes (9 MB) of RAM will be required (when
the second layer has just been computed, but the memory occupied by the first
layer has not been released yet). But wait, you also need to add the memory occu‐
pied by the CNN’s parameters! We computed earlier that it has 903,400 parame‐
ters, each using up 4 bytes, so this adds 3,613,600 bytes (about 3.6 MB). The total
RAM required is therefore (at least) 12,613,600 bytes (about 12.6 MB).

Lastly, let’s compute the minimum amount of RAM required when training the
CNN on a mini-batch of 50 images. During training TensorFlow uses backpropa‐
gation, which requires keeping all values computed during the forward pass until
the reverse pass begins. So we must compute the total RAM required by all layers
for a single instance and multiply that by 50. At this point, let’s start counting in
megabytes rather than bytes. We computed before that the three layers require
respectively 6, 3, and 1.5 MB for each instance. That’s a total of 10.5 MB per
instance, so for 50 instances the total RAM required is 525 MB. Add to that the
RAM required by the input images, which is 50 × 4 × 200 × 300 × 3 = 36 million
bytes (36 MB), plus the RAM required for the model parameters, which is about
3.6 MB (computed earlier), plus some RAM for the gradients (we will neglect this
since it can be released gradually as backpropagation goes down the layers during
the reverse pass). We are up to a total of roughly 525 + 36 + 3.6 = 564.6 MB, and
that’s really an optimistic bare minimum.

## 3. 
If your GPU runs out of memory while training a CNN, what are five things you could try to solve the problem?


If your GPU runs out of memory while training a CNN, here are five things you
could try to solve the problem (other than purchasing a GPU with more RAM):

- Reduce the mini-batch size.
- Reduce dimensionality using a larger stride in one or more layers.
- Remove one or more layers.
- Use 16-bit floats instead of 32-bit floats.
- Distribute the CNN across multiple devices.

## 4. 
Why would you want to add a max pooling layer rather than a convolutional layer with the same stride?


A max pooling layer has no parameters at all, whereas a convolutional layer has
quite a few (see the previous questions).

## 5. 
When would you want to add a local response normalization layer?


A local response normalization layer makes the neurons that most strongly acti‐
vate inhibit neurons at the same location but in neighboring feature maps, which
encourages different feature maps to specialize and pushes them apart, forcing
them to explore a wider range of features. It is typically used in the lower layers to
have a larger pool of low-level features that the upper layers can build upon.

## 6. 
Can you name the main innovations in AlexNet, compared to LeNet-5? What about the main innovations in GoogLeNet, ResNet, SENet, and Xception?


The main innovations in AlexNet compared to LeNet-5 are that it is much larger
and deeper, and it stacks convolutional layers directly on top of each other,
instead of stacking a pooling layer on top of each convolutional layer. The main
innovation in GoogLeNet is the introduction of inception modules, which make it
possible to have a much deeper net than previous CNN architectures, with fewer
parameters. ResNet’s main innovation is the introduction of skip connections,
which make it possible to go well beyond 100 layers. Arguably, its simplicity and
consistency are also rather innovative. SENet’s main innovation was the idea of
using an SE block (a two-layer dense network) after every inception module in
an inception network or every residual unit in a ResNet to recalibrate the relative
importance of feature maps. Finally, Xception’s main innovation was the use of
depthwise separable convolutional layers, which look at spatial patterns and
depthwise patterns separately.

## 7. 
What is a fully convolutional network? How can you convert a dense layer into a convolutional layer?


Fully convolutional networks are neural networks composed exclusively of con‐
volutional and pooling layers. FCNs can efficiently process images of any width
and height (at least above the minimum size). They are most useful for object
detection and semantic segmentation because they only need to look at the image
once (instead of having to run a CNN multiple times on different parts of the
image). If you have a CNN with some dense layers on top, you can convert these
dense layers to convolutional layers to create an FCN: just replace the lowest
dense layer with a convolutional layer with a kernel size equal to the layer’s input
size, with one filter per neuron in the dense layer, and using "valid" padding.
Generally the stride should be 1, but you can set it to a higher value if you want.
The activation function should be the same as the dense layer’s. The other dense
layers should be converted the same way, but using 1 × 1 filters. It is actually pos‐
sible to convert a trained CNN this way by appropriately reshaping the dense lay‐
ers’ weight matrices.

## 8. 
What is the main technical difficulty of semantic segmentation?


The main technical difficulty of semantic segmentation is the fact that a lot of the
spatial information gets lost in a CNN as the signal flows through each layer,
especially in pooling layers and layers with a stride greater than 1. This spatial
information needs to be restored somehow to accurately predict the class of each
pixel.

## 9. 
Build your own CNN from scratch and try to achieve the highest possible accu‐ racy on MNIST.

The following model uses 2 convolutional layers, followed by 1 pooling layer, then dropout 25%, then a dense layer, another dropout layer but with 50% dropout, and finally the output layer. It reaches about 99.2% accuracy on the test set. This places this model roughly in the top 20% in the [MNIST Kaggle competition](https://www.kaggle.com/c/digit-recognizer/) (if we ignore the models with an accuracy greater than 99.79% which were most likely trained on the test set, as explained by Chris Deotte in [this post](https://www.kaggle.com/c/digit-recognizer/discussion/61480)). Can you do better? To reach 99.5 to 99.7% accuracy on the test set, you need to add image augmentation, batch norm, use a learning schedule such as 1-cycle, and possibly create an ensemble.

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.
X_test = X_test / 255.
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [3]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, padding="same", activation="relu"),
    keras.layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

2024-03-18 17:08:10.610928: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 17:08:10.612434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-18 17:08:10.612911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-18 17:08:10.613234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Epoch 1/10


2024-03-18 17:08:14.693479: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8401


1719/1719 [==============================] - 32s 17ms/step - loss: 0.1968 - accuracy: 0.9396 - val_loss: 0.0443 - val_accuracy: 0.9868
Epoch 2/10
1719/1719 [==============================] - 30s 18ms/step - loss: 0.0790 - accuracy: 0.9761 - val_loss: 0.0465 - val_accuracy: 0.9880
Epoch 3/10
1719/1719 [==============================] - 31s 18ms/step - loss: 0.0618 - accuracy: 0.9809 - val_loss: 0.0368 - val_accuracy: 0.9900
Epoch 4/10
1719/1719 [==============================] - 31s 18ms/step - loss: 0.0504 - accuracy: 0.9849 - val_loss: 0.0385 - val_accuracy: 0.9904
Epoch 5/10
1719/1719 [==============================] - 31s 18ms/step - loss: 0.0416 - accuracy: 0.9865 - val_loss: 0.0322 - val_accuracy: 0.9930
Epoch 6/10
1719/1719 [==============================] - 32s 19ms/step - loss: 0.0361 - accuracy: 0.9883 - val_loss: 0.0440 - val_accuracy: 0.9892
Epoch 7/10
1719/1719 [==============================] - 34s 20ms/step - loss: 0.0337 - accuracy: 0.9891 - val_loss: 0.0354 - val_accura

[0.026768291369080544, 0.9925000071525574]

## 10. 
Use transfer learning for large image classification, going through these steps:

a. Create a training set containing at least 100 images per class. For example, you
could classify your own pictures based on the location (beach, mountain, city,
etc.), or alternatively you can use an existing dataset (e.g., from TensorFlow
Datasets).

b. Split it into a training set, a validation set, and a test set.

c. Build the input pipeline, including the appropriate preprocessing operations,
and optionally add data augmentation.

d. Fine-tune a pretrained model on this dataset.


## 11. 
Go through TensorFlow’s Style Transfer tutorial. It is a fun way to generate art using Deep Learning.